### تفسیر و توضیح مدل

In [1]:
#pip install tf-explain


In [3]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import numpy as np
%matplotlib inline

In [4]:
path = '../data/flowers'
img = tf.keras.preprocessing.image.load_img('../data/flowers/daisy/100080576_f52e8ee070_n.jpg', target_size=(150, 150))
img = tf.keras.preprocessing.image.img_to_array(img)
data = img.reshape(1,*img.shape)

In [7]:
img_gen_train = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.5)

train_data = img_gen_train.flow_from_directory('../data/flowers',
                          batch_size = 8,
                                  seed =42,
                                  target_size = (150,150),
                        subset = 'training' 
                                 )

val_data = img_gen_train.flow_from_directory('../data/flowers',
                          batch_size = 8,
                                  seed =42,
                                  target_size = (150,150),
                                subset = 'validation' )

Found 2162 images belonging to 5 classes.
Found 2161 images belonging to 5 classes.


In [20]:
(x,y) = val_data.next()
(x,y) = val_data.next()
for el, label in zip(x,y):
    print(el.shape)
    print(label.shape)
    print(np.argmax(label))

validation_class_zero = (np.array([
    el for el, label in zip(x,y)
    if np.all(np.argmax(label) == 0)
][0:5]), None)


(150, 150, 3)
(5,)
0
(150, 150, 3)
(5,)
3
(150, 150, 3)
(5,)
2
(150, 150, 3)
(5,)
0
(150, 150, 3)
(5,)
3
(150, 150, 3)
(5,)
2
(150, 150, 3)
(5,)
3
(150, 150, 3)
(5,)
3


In [21]:
validation_class_zero

(array([[[[0.29803923, 0.09803922, 0.        ],
          [0.36862746, 0.13333334, 0.        ],
          [0.42352945, 0.15294118, 0.        ],
          ...,
          [0.02745098, 0.05490196, 0.08627451],
          [0.04705883, 0.0627451 , 0.10980393],
          [0.10980393, 0.10196079, 0.18431373]],
 
         [[0.31764707, 0.10588236, 0.        ],
          [0.3803922 , 0.14509805, 0.00392157],
          [0.43529415, 0.16470589, 0.00392157],
          ...,
          [0.0509804 , 0.07058824, 0.09411766],
          [0.05882353, 0.06666667, 0.11764707],
          [0.1137255 , 0.10588236, 0.18823531]],
 
         [[0.36078432, 0.1254902 , 0.        ],
          [0.43137258, 0.16862746, 0.00392157],
          [0.4784314 , 0.18823531, 0.        ],
          ...,
          [0.16862746, 0.14509805, 0.18431373],
          [0.1137255 , 0.10196079, 0.14509805],
          [0.12156864, 0.10588236, 0.16470589]],
 
         ...,
 
         [[0.6901961 , 0.52156866, 0.5568628 ],
          [0.69019

In [16]:
from tf_explain.callbacks import ActivationsVisualizationCallback
import tf_explain

In [14]:
callbacks = [
    ActivationsVisualizationCallback(
        validation_data= (x,y),
        layers_name=["activation_1"],
        output_dir='imgs/',
    ),
]

In [10]:
base_model = keras.applications.MobileNet( weights= 'imagenet',
                                         input_shape = (150,150,3),
                                         include_top = False)
def create_transfer_learning_model():
    base_model.trainable = False
    inputs = keras.layers.Input(shape = (150,150,3))
    x = base_model(inputs,training = False)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128,activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(5,activation='softmax' )(x)
    final_model = keras.Model(inputs,outputs)
    return final_model


model = create_transfer_learning_model()
model.compile(optimizer = 'adam',
             loss ='categorical_crossentropy',metrics = ['accuracy'])

In [22]:
model.fit(train_data,epochs=1,
          callbacks=[tf_explain.callbacks.VanillaGradientsCallback(validation_class_zero, class_index=0)])

271/271 [==============================] - 30s 109ms/step - loss: 0.2616 - accuracy: 0.9075


### روش smooth-grad

In [36]:
(x,y)  = val_date.next()



In [49]:
model = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=True)

# Load a sample image (or multiple ones)
img = tf.keras.preprocessing.image.load_img('imgs/cat.jpg', target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)
data = ([img], None)

# Start explainer
explainer = GradCAM()
grid = explainer.explain(data, model, class_index=281)  # 281 is the tabby cat index in ImageNet

explainer.save(grid, ".", "grad_cam.png")

TypeError: 'NoneType' object is not callable

In [5]:
    model = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=True)

    img = tf.keras.preprocessing.image.load_img('imgs/cat.jpg', target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)

    model.summary()
    data = ([img], None)

img

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

array([[[242., 237., 255.],
        [225., 222., 243.],
        [195., 194., 212.],
        ...,
        [232., 232., 234.],
        [232., 232., 234.],
        [232., 232., 234.]],

       [[242., 237., 255.],
        [223., 220., 241.],
        [193., 192., 210.],
        ...,
        [232., 232., 234.],
        [232., 232., 234.],
        [232., 232., 234.]],

       [[241., 236., 255.],
        [221., 218., 239.],
        [190., 189., 207.],
        ...,
        [232., 232., 234.],
        [232., 232., 234.],
        [232., 232., 234.]],

       ...,

       [[122., 124., 123.],
        [122., 124., 123.],
        [122., 124., 123.],
        ...,
        [211., 211., 211.],
        [211., 211., 211.],
        [211., 211., 211.]],

       [[119., 121., 120.],
        [119., 121., 120.],
        [119., 121., 120.],
        ...,
        [211., 211., 211.],
        [211., 211., 211.],
        [211., 211., 211.]],

       [[121., 123., 122.],
        [121., 123., 122.],
        [121., 1

In [54]:
from tf_explain.core.smoothgrad import SmoothGrad
   



In [55]:
    tabby_cat_class_index = 281
    explainer = SmoothGrad()
    # Compute SmoothGrad on VGG16
    grid = explainer.explain(data, model, tabby_cat_class_index, 20, 1.0)
    explainer.save(grid, ".", "smoothgrad.png")


In [58]:
model.fit(train_data,batch_size=16, epochs=1)

271/271 [==============================] - 20s 75ms/step - loss: 1.3295 - accuracy: 0.7169


In [59]:
val_date.class_indices

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [63]:
img = tf.keras.preprocessing.image.load_img('../data/flowers/daisy/100080576_f52e8ee070_n.jpg', target_size=(150, 150))
img = tf.keras.preprocessing.image.img_to_array(img)
data = ([img], None)

In [67]:
tabby_cat_class_index = 0
explainer = SmoothGrad()
# Compute SmoothGrad on VGG16
grid = explainer.explain(data, model, tabby_cat_class_index)
explainer.save(grid, ".", "smoothgrad.png")